In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
from DFRSystem import *
import pandas as pd

from Loss import ber_loss


import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

generating_data = False

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    #device = torch.device('cuda:0')
    #device = torch.device("cuda") 
    device = torch.device("cpu")
else:
    device = torch.device("cpu")
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")
print(device)

/home/shiya/anaconda3/envs/lava_test/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/shiya/anaconda3/envs/lava_test/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done
cpu


In [2]:
def pre_processing(train_input, train_label):
    idx_p = 10
    begin = 0 # N_total_frame * N_sync_frame
    
    # label index for data
    train_input_df = pd.DataFrame(train_input, columns = ['1','2', '3', '4'])
    #train_input_df['L1_idx'] = train_input_df.index % idx_p

    # label index for label
    train_label_df = pd.DataFrame(train_label, columns = ['L1','L2'])
    train_label_df['L1_idx'] = train_label_df.index % idx_p
    
    # split training and testing data
    test_input_df, test_label_df = train_input_df.iloc[75* 80 + 1:, :], train_label_df.iloc[75* 80 + 1:, :]
    train_input_df, train_label_df = train_input_df.iloc[:75* 80 + 1, :], train_label_df.iloc[:75* 80 + 1, :]

    # group by 
    #mapping = train_label_df.loc[begin:, :].groupby(by='L1_idx').mean().reset_index().loc[:, ['L1', 'L2', 'L1_idx']] 
    
    #train_input_df = pd.merge(train_input_df, mapping, how='left', on='L1_idx')

    #train_input_df = pd.get_dummies(train_input_df, prefix=['L'], columns=['L1_idx'])

    train_input_df = train_input_df.loc[begin:, :]


    #print(train_input_df.head())
    
    # testing data
    # group by
    #test_input_df = test_input_df.merge(mapping, how = 'left', on='L1_idx')

    #test_input_df = pd.get_dummies(test_input_df, prefix=['L'], columns=['L1_idx'])

    #print(test_input_df.head())

    train_input = train_input_df.to_numpy()
    test_input = test_input_df.to_numpy()
    
    train_label = train_label_df.drop(['L1_idx'], axis=1).to_numpy()
    test_label = test_label_df.drop(['L1_idx'], axis=1).to_numpy()
    
    #(train_input.shape)
    #print(test_input.shape)
    
    
    return train_input, train_label, test_input, test_label

In [3]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : 1.2,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,   
            
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #slayer.block.cuba.Input(neuron_params),
                slayer.block.cuba.Recurrent(neuron_params, input_size, 8, weight_norm=True, delay=True, weight_scale=1.0),
                #slayer.block.cuba.Dense(neuron_params, input_size, 8, weight_norm=True, delay=True, weight_scale=1.0),
                slayer.block.cuba.Affine(neuron_params, 8, 1, weight_norm=True),
                #slayer.block.cuba.Dense(neuron_params, 8, 1, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 128, output_size, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, input_size, 64, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 64, 128, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 128, output_size, weight_scale=2, weight_norm=True)
                #slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                #slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5)
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        #self.fc1 = nn.Linear(input_size, 2)
        self.fc1 = nn.Linear(100, 8)
        self.conv1 = torch.nn.Conv1d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=1)
        # self.conv2 = torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc2 = nn.Linear(16 * 4 + 8, 2)
        #self.act = nn.Tanh()
        self.act = nn.ReLU()

    def forward(self,x, x1, x2):
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.act(x)
        x2 = x2.view(x2.shape[0], 6, 4)
        x2 = self.conv1(x2)
        x2 = self.act(x2)
        #x2 = self.conv2(x2)
        #x2 = self.act(x2)
        x2 = x2.flatten(start_dim=1)
        x = torch.cat((x, x2), dim=1)
        x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc2(x)
        return x

In [4]:
def test(test_loader, net_snn, net_dnn, rc, num_frame = 19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().clone().detach().numpy())
        #target = torch.tensor(target).float()
        #input = torch.tensor(input).float()
        #input1 = torch.tensor(input1).float()
        #input2 = torch.tensor(input2).float
        
        target = target.float().to(device)
        input = input.float().to(device)
        input1 = input1.float().to(device)
        input2 = input2.float().to(device)
    
        output = net_snn(input)
        
        output = net_dnn(output, input1, input2).cpu().clone().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    return rc.my_new_test(predict_time, target_time)


def test_teacher(test_loader, model, rc, nb_inputs, num_frame=19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().clone().detach().numpy())
        #target = torch.tensor(target).float()
        #input = torch.tensor(input).float()
        #input1 = torch.tensor(input1).float()
        #input2 = torch.tensor(input2).float()
        target = target.float().to(device)
        input = input.float().to(device)
        input1 = input1.float().to(device)
        input2 = input2.float().to(device)
        
        input2 = input2.view(input2.shape[0], nb_inputs, 4)
        prev_out = 0.1 * torch.zeros(input2.size(0), model.n_hidden).to(device)
        output = model(input2, prev_out)[0].cpu().clone().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    #print("testing teacher MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(labels)))
    #print("testing teacher MSE 0: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 0]), torch.tensor(labels[:, 0])))
    #print("testing teacher MSE 1: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 1]), torch.tensor(labels[:, 1])))
    #print(all_output.shape, labels.shape)
    #print("percent teacher loss: ", np.mean(np.abs(all_output - labels) / (np.abs(labels) + 1e-6)))
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    return rc.my_new_test(predict_time, target_time)

In [5]:
def train_dnn(trainloader, testloader, net, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    best_ber = 10000000000.0
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer.zero_grad()
    
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net.n_hidden).to(device)
            output = net(x3_local, prev_out)[0]
            
            loss = loss_fn(output, y_local)
            
            loss.backward()
            optimizer.step()
            loss_hist.append(loss.item())
        
        ber = test_teacher(test_loader, net, rc, nb_inputs, num_frame=19) 
        if ber < best_ber:
            best_ber = ber
            torch.save(net.state_dict(), './teacher.pt')
    return best_ber

In [6]:
def mutual_loss(teacher_output, student_output, alpha, target, mode=True):
    # student loss 1
    loss_fn = torch.nn.L1Loss()
    
    #compare 
    loss_t = loss_fn(teacher_output, target)
    loss_s = loss_fn(student_output, target)
    
    loss1 = loss_fn(student_output, target)
    loss2 = loss_fn(student_output, teacher_output)
    if mode:
        if loss_t >= loss_s:
            return loss1
        else:
            return loss1 + alpha * loss2 #loss1 + alpha * loss2
    else:
        return loss1 + alpha * loss2
    
    
def mutual_loss1(teacher_output, student_output, alpha, target, mode=True):
    # student loss 1
    loss_fn = torch.nn.L1Loss(reduction='none')
    
    #compare 
    loss_t = loss_fn(teacher_output, target)
    loss_s = loss_fn(student_output, target)
    
    A_p = loss_t >= loss_s
    A_n = ~A_p
    
    loss1 = loss_fn(student_output, target)
    loss2 = loss_fn(student_output, teacher_output)
    
    # print(loss1.shape, loss2.shape, A_p)
    
    loss = A_p * loss1 + A_n * (loss1 + alpha * loss2)
    
    return torch.sum(loss)

def med_loss1(student_output, teacher_output, med_student, med_teacher, target, alpha=0.05):
    med_loss_fn = torch.nn.L1Loss(reduction='none')
    
    #compare 
    loss_t = loss_fn(teacher_output, target)
    loss_s = loss_fn(student_output, target)
    
    A_p = loss_t < loss_s
    
    loss = A_p * alpha * med_loss_fn(med_student, med_teacher)
    return loss
    
    
def KD_train(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, teacher_path, nb_inputs, lr=2e-3, nb_epochs=10):
    net_teacher.load_state_dict(torch.load(teacher_path))
    net_teacher.eval()
    test_teacher(testloader, net_teacher, rc, nb_inputs, num_frame=19)
    
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=200, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    
    min_ber = 10000000000000000.0
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            
            # snn model 
            output = net_snn(x1_local)
            output = net_dnn(output, x2_local, x3_local)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
            teacher_output, _ = net_teacher(x3_local, prev_out)
            
            student_loss = mutual_loss(teacher_output.clone().detach(), output, alpha=0.0, target=y_local, mode=True)
            #student_loss = loss_fn(output, y_local)
            student_loss.backward()

            optimizer_student.step()
        
        test_ber = test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        min_ber = min(min_ber, test_ber)
        #test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)  
    return min_ber

def mutual_train(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    optimizer_teacher = torch.optim.Adam(net_teacher.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=100, gamma=0.1)
    scheduler_teacher = torch.optim.lr_scheduler.StepLR(optimizer_teacher, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    min_ber = 10000000000000000.0
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            optimizer_teacher.zero_grad()
            
            # snn model 
            output = net_snn(x1_local)
            output = net_dnn(output, x2_local, x3_local)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
            teacher_output, _ = net_teacher(x3_local, prev_out)
            
            student_loss = mutual_loss(teacher_output.clone().detach(), output, alpha=0.05, target=y_local, mode=True)
            teacher_loss = mutual_loss(output.clone().detach(), teacher_output, alpha=0.01, target=y_local, mode=False)
            student_loss.backward()
            teacher_loss.backward()
            optimizer_student.step()
            optimizer_teacher.step()
            loss_hist.append(student_loss.item() + teacher_loss.item())
        
        test_ber = test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)
        min_ber = min(min_ber, test_ber)
    return min_ber


def mutual_train_med(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    optimizer_teacher = torch.optim.Adam(net_teacher.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=30, gamma=0.1)
    scheduler_teacher = torch.optim.lr_scheduler.StepLR(optimizer_teacher, step_size=30, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    min_ber = 10000000000000000.0
    min_teacher_ber = 10000000000000000.0
    
    spike_med_size = 1
    rc_med_size = 32
    
    med_loss_fn = torch.nn.L1Loss()
    med_mapping = torch.rand(rc_med_size, spike_med_size, requires_grad = False).to(device)
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            optimizer_teacher.zero_grad()
            
            # snn model 
            output_spike_med = net_snn(x1_local)
            output = net_dnn(output_spike_med, x2_local, x3_local)
            output_spike_med_mean = torch.mean(output_spike_med, dim=-1)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
            teacher_output, med = net_teacher(x3_local, prev_out)
            
            student_loss = mutual_loss1(teacher_output.clone().detach(), output, alpha=0.10, target=y_local, mode=True)
            teacher_loss = mutual_loss1(output.clone().detach(), teacher_output, alpha=0.05, target=y_local, mode=True)
           
            # output_spike_med_flatten = torch.flatten(output_spike_med, start_dim=1)
            # map_spike_med = torch.matmul(output_spike_med_mean, med_mapping)
            map_spike_med = torch.matmul(med, med_mapping)
            med_loss = med_loss_fn(output_spike_med_mean, map_spike_med.clone().detach())
            
            student_loss += 0.1 * med_loss
            
            student_loss.backward()
            teacher_loss.backward()
            
            optimizer_student.step()
            optimizer_teacher.step()
            loss_hist.append(student_loss.item() + teacher_loss.item())
        
        test_ber = test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        test_teacher_ber = test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)
        min_ber = min(min_ber, test_ber)
        min_teacher_ber = min(min_teacher_ber, test_teacher_ber)
    return min_ber, min_teacher_ber


def mutual_train_1(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    #params = net_dnn.parameters()
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    optimizer_teacher = torch.optim.Adam(net_teacher.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=30, gamma=0.1)
    scheduler_teacher = torch.optim.lr_scheduler.StepLR(optimizer_teacher, step_size=30, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    min_ber = 10000000000000000.0
    min_teacher_ber = 10000000000000000.0
    
    spike_med_size = 100
    rc_med_size = 32
    
    med_loss_fn = torch.nn.L1Loss()
    med_mapping = torch.rand(spike_med_size, rc_med_size, requires_grad = False).to(device)
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            optimizer_teacher.zero_grad()
            
            # snn model 
            output_spike_med = net_snn(x1_local)
            output = net_dnn(output_spike_med, x2_local, x3_local)
            student_loss = med_loss_fn(output, y_local)
            
            student_loss.backward()
            
            optimizer_student.step()
        
        test_ber = test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        #test_teacher_ber = test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)
        min_ber = min(min_ber, test_ber)
        #min_teacher_ber = min(min_teacher_ber, test_teacher_ber)
    return min_ber

def mutual_train_2(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    optimizer_teacher = torch.optim.Adam(net_teacher.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=30, gamma=0.1)
    scheduler_teacher = torch.optim.lr_scheduler.StepLR(optimizer_teacher, step_size=30, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    min_ber = 10000000000000000.0
    min_teacher_ber = 10000000000000000.0
    
    spike_med_size = 100
    rc_med_size = 32
    
    med_loss_fn = torch.nn.L1Loss()
    med_mapping = torch.rand(spike_med_size, rc_med_size, requires_grad = False).to(device)
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            optimizer_teacher.zero_grad()
            
            # snn model 
            output_spike_med = net_snn(x1_local)
            output = net_dnn(output_spike_med, x2_local, x3_local)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
            teacher_output, med = net_teacher(x3_local, prev_out)
            
            student_loss = mutual_loss1(teacher_output.clone().detach(), output, alpha=0.10, target=y_local, mode=True)
            teacher_loss = mutual_loss1(output.clone().detach(), teacher_output, alpha=0.05, target=y_local, mode=True)
            
            student_loss.backward()
            teacher_loss.backward()
            optimizer_student.step()
            optimizer_teacher.step()
        
        test_ber = test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        #test_teacher_ber = test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)
        min_ber = min(min_ber, test_ber)
        #min_teacher_ber = min(min_teacher_ber, test_teacher_ber)
    return min_ber
        

In [7]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [0]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]

overall_mu_ber = 0.0
overall_teacher_ber = 0.0
overall_base_ber = 0.0
overall_nopath_ber = 0.0
cnt = 0

# except 2
packets =[13, 14, 16, 17, 18, 19, 20]  # [11, 13, 14, 16,]
for i in packets:
    print(i)
    rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
            N_reservoir=16,
            spectral_radius=0.2,
            sparsity=0.4,
            noise=1e-6,
            lut_activation=False,  # True,
            tanh_lut=tanh_lut,
            input_scale=25,  #40, #50, # 25,
            reservoir_input_scale = 8,  #4,  #5,
            show_wout=False,
            output_folder= output_folder,
            debug=debug,
            use_fpga= None,
            sock= None,  # usock
            addr = None) # addr

    train_input, train_label, test_input, test_label = rc.run()
    train_max = np.max(train_input)
    train_min = np.min(train_input)
    train_mean = np.mean(train_input)
    train_std = np.std(train_input)
    print(train_max, train_min, train_mean, train_std)
   
    #RC_test_input = np.load('gt_test_input_1.npy')
    #RC_train_input = np.load('gt_train_input_1.npy')
    #RC_train_label = np.load('gt_train_label_1.npy')
 
    #print(RC_test_input.shape, test_input.shape)
    #print("test_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_test_input), torch.tensor(test_input)))

    #print("train_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_input), torch.tensor(train_input)))

    #print("train_label_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_label), torch.tensor(train_label)))

    train_mean = np.mean(train_input)
    train_std = np.std(train_input)

    train_input = (train_input - train_mean) / train_std
    test_input = (test_input - train_mean) / train_std
    train_label = 1.0 * train_label
    
    train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)
    
    
    nb_inputs  = 6
    nb_steps  = 100

    batch_size = 128

    import scipy.io

    from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset, RegTorchSeasonalityLinearSpikingDataset
    train_data = RegTorchSeasonalityLinearSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
    test_data = RegTorchSeasonalityLinearSpikingDataset(test_input, test_label, nb_inputs, nb_steps)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
    

    # KD learning
    #net_snn = Network(nb_inputs * 4, 2).to(device)
    #net_dnn = DNNNetwork(1 * nb_steps + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
    #net_rc = TOriFloatDFRSystem(n_hidden=16, n_fc=64)
    
    #teacher_min_ber = train_dnn(train_loader, test_loader, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    #KD_min_ber = KD_train(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, './teacher.pt', nb_inputs, lr=1e-3, nb_epochs=100)
    
    # mutual learning
    net_snn = Network(nb_inputs * 4, 2).to(device)
    net_dnn = DNNNetwork(1 * nb_steps + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
    net_rc = TOriFloatDFRSystem(n_hidden=16, n_fc=32).to(device)
    min_mu_ber, min_teacher_ber = mutual_train_med(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    overall_mu_ber += min_mu_ber
    overall_teacher_ber += min_teacher_ber
    print(min_mu_ber, min_teacher_ber)
    """
    #base model
    net_snn = Network(nb_inputs * 4, 2).to(device)
    net_dnn = DNNNetwork(1 * nb_steps + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
    min_base_ber = mutual_train_1(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    overall_base_ber += min_base_ber
    
    # no path
    net_snn = Network(nb_inputs * 4, 2).to(device)
    net_dnn = DNNNetwork(1 * nb_steps + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
    net_rc = TOriFloatDFRSystem(n_hidden=16, n_fc=32).to(device)
    min_nopath_ber = mutual_train_2(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    overall_nopath_ber += min_nopath_ber
    
    cnt += 1
    
    print(min_base_ber, min_mu_ber, min_teacher_ber, min_nopath_ber)
    """

print("ML SNN: ", overall_mu_ber / len(packets))
print("teacher: ", overall_teacher_ber / len(packets))
print("base: ", overall_base_ber / len(packets))
print("nopath: ", overall_nopath_ber / len(packets))

13
0.03388886815819279 -0.033966198217177786 -1.3872054205025038e-05 0.004555123840129171


TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'